In [1]:
# Seccion de importacion de librerias
import pandas as pd
import datetime
import firebase_admin
from firebase_admin import credentials, firestore
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import datetime
from google.cloud.firestore_v1._helpers import DatetimeWithNanoseconds
import datetime
from collections import defaultdict

In [2]:
# Replace 'path/to/serviceAccountKey.json' with the path to your downloaded service account JSON file.
cred = credentials.Certificate('credentials/serviceAccountKey.json')

# Initialize the Firebase app with the credentials.
firebase_admin.initialize_app(cred)

# Initialize Firestore DB
db = firestore.client()

## Type 1 - App's Telemetry: How many bugs have been reported in the past week?

In [3]:
# Acceder a la colección 'user_actions'
feedbacks_ref = db.collection('feedbacks')

# Obtener todos los documentos en la colección
docs = feedbacks_ref.stream()

In [4]:
# Iterar sobre los documentos y mostrar su contenido
for doc in docs:
    print(f"Document ID: {doc.id}")
    user_data = doc.to_dict()
    for bug_id, bug_data in user_data.items():
        print(f"  Bug ID: {bug_id}")
        print(f"    Data: {bug_data}")

Document ID: pauladaza600@gmail.com
  Bug ID: 3
    Data: {'description': 'The nearby properties notification doesn’t trigger when close to the university.', 'date': DatetimeWithNanoseconds(2024, 11, 29, 23, 40, 19, 107087, tzinfo=datetime.timezone.utc)}
  Bug ID: 1
    Data: {'description': 'The price filter doesn’t update results correctly when applied multiple times.', 'date': DatetimeWithNanoseconds(2024, 11, 25, 16, 1, 34, 359000, tzinfo=datetime.timezone.utc)}
  Bug ID: 2
    Data: {'description': 'Property images take too long to load, even with a fast connection.', 'date': DatetimeWithNanoseconds(2024, 11, 29, 23, 36, 37, 13338, tzinfo=datetime.timezone.utc)}
Document ID: tamaiothais@gmail.com
  Bug ID: 1
    Data: {'description': 'The app crashes unexpectedly when trying to upload an image from the landlord profile.', 'date': DatetimeWithNanoseconds(2024, 12, 2, 23, 0, 54, 160000, tzinfo=datetime.timezone.utc)}
  Bug ID: 2
    Data: {'description': 'The contact landlord button

In [5]:
# Obtener todos los documentos en la colección nuevamente
docs = feedbacks_ref.stream()  # Volver a obtener los documentos

# Obtener la fecha actual y calcular la fecha de hace 7 días
current_date = datetime.datetime.utcnow()
one_week_ago = current_date - datetime.timedelta(days=7)

# Contador para bugs reportados en la última semana
bug_count = 0

for doc in docs:
    user_data = doc.to_dict()
    
    # Iterar sobre los datos del usuario (bugs)
    for bug_id, bug_data in user_data.items():
        # Extraer la fecha del bug
        bug_date = bug_data.get('date')
        
        # Convertir DatetimeWithNanoseconds a datetime si es necesario
        if isinstance(bug_date, DatetimeWithNanoseconds):
            bug_date = bug_date.replace(tzinfo=None)  # Eliminar zona horaria para comparación
        
        # Comprobar si la fecha está dentro de la última semana
        if isinstance(bug_date, datetime.datetime) and one_week_ago <= bug_date <= current_date:
            bug_count += 1

# Imprimir el número total de bugs reportados en la última semana
print(f"Total Number of Bugs Reported in the Past Week: {bug_count}")

Total Number of Bugs Reported in the Past Week: 4


## Type 5 - Mixed Type Questions: What are the top 3 features with the most bug reports?

In [ ]:
from langflow.load import run_flow_from_json
import pandas as pd

# Ruta al archivo JSON del flujo (absoluta o relativa según tu estructura)
flow_path = "/Users/thaistamaio/Library/Mobile Documents/com~apple~CloudDocs/universidad/9) Noveno semestre/Móviles/Analytics/flow/flow.json"

# Lista de bugs a procesar
bugs = [
    "The application crashes when filtering properties by price range.",
    "The contact landlord button redirects to an empty screen.",
    "Users cannot save properties while offline.",
    "The notifications do not appear when close to the university.",
    "The app freezes when trying to upload a property image."
]

# Configuración original del flujo
TWEAKS_TEMPLATE = {
    "Prompt-kDXu3": {
        "Document": "",
        "Question": "",
        "template": "Answer user's questions based on the document below:\n\n---\n\n{Document}\n\n---\n\nQuestion:\nClassify the following bug report into one of the existing functionalities of the application. The functionalities are as follows:\n- Crear una nueva propiedad\n- Filtrar todas las propiedades\n- Contactar a un landlord\n- Iniciar sesión\n- Cerrar sesión\n- Reportar un bug\n- Visualizar detalles de propiedades\n- Guardar una propiedad\n- Recibir notificaciones\n\nIf it is not a bug, classify it as \"Basura\".\n\nONLY RETURN THE FUNCTIONALITY\n\nBug report: {Question}\n\nAnswer:\n"
    },
    "ChatInput-A0qNJ": {
        "files": "",
        "input_value": "",
        "sender": "User",
        "sender_name": "User",
        "session_id": "",
        "should_store_message": True
    },
    "ChatOutput-HtcQ8": {
        "data_template": "{text}",
        "input_value": "",
        "sender": "Machine",
        "sender_name": "AI",
        "session_id": "",
        "should_store_message": True
    },
    "OpenAIModel-Ncboq": {
        "api_key": "",
        "input_value": "",
        "json_mode": False,
        "max_tokens": None,
        "model_kwargs": {},
        "model_name": "gpt-4o",
        "openai_api_base": "",
        "output_schema": {},
        "seed": 1,
        "stream": False,
        "system_message": "",
        "temperature": 0.1
    },
    "ParseData-ZXPg0": {
        "sep": "\n",
        "template": "{text}"
    },
    "File-XOcmZ": {
        "path": "/Users/thaistamaio/Library/Mobile Documents/com~apple~CloudDocs/universidad/9) Noveno semestre/Móviles/Analytics/flow/FuncionalidadesAPP.pdf",
        "silent_errors": False
    }
}

# Inicializar una lista para almacenar los resultados
results = []

# Iterar sobre cada bug y procesarlo con LangFlow
for bug in bugs:
    print(f"Processing bug: {bug}")
    try:
        # Clonar TWEAKS y ajustar para el bug actual
        tweaks = TWEAKS_TEMPLATE.copy()
        tweaks["ChatInput-A0qNJ"]["input_value"] = f"Classify the following bug report: '{bug}'"

        # Ejecutar el flujo de LangFlow
        result = run_flow_from_json(
            flow=flow_path,
            input_value=bug,
            tweaks=tweaks
        )

        # Almacenar el resultado
        results.append({"Bug Report": bug, "Classification": result})

    except Exception as e:
        print(f"Error processing bug '{bug}': {e}")
        results.append({"Bug Report": bug, "Classification": "Error"})

# Crear un DataFrame con los resultados
df_results = pd.DataFrame(results)

# Mostrar el DataFrame
print("\nResults DataFrame:")
print(df_results)

# Guardar los resultados en un archivo CSV (opcional)
df_results.to_csv("bug_classifications.csv", index=False)

ImportError: cannot import name 'field_validator' from 'pydantic' (/Users/thaistamaio/opt/anaconda3/lib/python3.9/site-packages/pydantic/__init__.cpython-39-darwin.so)

In [ ]:
import pandas as pd
import re

# Cargar el archivo CSV
df = pd.read_csv('bug_classifications.csv')

# Función para extraer el texto dentro de 'data={'text':', tomar solo el texto antes de la primera coma, y eliminar las comillas dobles
def extract_and_clean_text(classification):
    match = re.search(r"data=\{'text': '(.*?)'", classification)
    if match:
        # Obtener el texto antes de la primera coma y eliminar las comillas dobles
        cleaned_text = match.group(1).split(",")[0].replace('"', '').strip()
        return cleaned_text
    return ''

# Aplicamos la función de extracción y limpieza
df['Cleaned Text'] = df['Classification'].apply(extract_and_clean_text)

# Guardamos el archivo resultante
output_cleaned_file_path = 'cleaned_output_file.csv'
df[['Bug Report', 'Cleaned Text']].to_csv(output_cleaned_file_path, index=False)

# Mostrar las primeras filas del resultado
df[['Bug Report', 'Cleaned Text']].head()


## Type 2 - Direct User Improvement: How many tenants have favorited the listings of a specific landlord in the last week?

In [8]:
# Obtener la fecha actual y calcular la fecha de hace 7 días
current_date = datetime.datetime.utcnow()
one_week_ago = current_date - datetime.timedelta(days=7)

# Paso 1: Crear DataFrame de 'offers'
offers_ref = db.collection('offers')
offers_docs = offers_ref.stream()

offers_data = []
for doc in offers_docs:
    offers_dict = doc.to_dict()
    for offer_id, offer_data in offers_dict.items():
        offer_data['offer_id'] = offer_id
        offers_data.append(offer_data)

df_offers = pd.DataFrame(offers_data)

# Paso 2: Crear DataFrame de 'property_saved_by'
property_saved_by_ref = db.collection('property_saved_by')
property_saved_docs = property_saved_by_ref.stream()

saved_properties = []
for doc in property_saved_docs:
    property_id = doc.id  # El ID del documento es el ID de la propiedad
    saved_data = doc.to_dict()  # Datos de tenants que guardaron la propiedad
    for tenant_id, save_date in saved_data.items():
        # Convertir DatetimeWithNanoseconds a datetime si es necesario
        if isinstance(save_date, DatetimeWithNanoseconds):
            save_date = save_date.replace(tzinfo=None)
        saved_properties.append({
            "property_id": int(property_id),  # Aseguramos que el ID sea entero para el merge
            "tenant_id": tenant_id,
            "save_date": save_date
        })

df_saved_properties = pd.DataFrame(saved_properties)

# Paso 3: Filtrar registros por fecha
df_saved_properties = df_saved_properties[df_saved_properties['save_date'] >= one_week_ago]

# Paso 4: Combinar con 'df_offers' para mapear landlords
df_combined = pd.merge(df_saved_properties, df_offers, left_on="property_id", right_on="id_property", how="inner")

# Paso 5: Agrupar por landlord (user_id) y contar
result = df_combined.groupby("user_id").size().reset_index(name="favorites_count")

In [9]:
print(result)

                 user_id  favorites_count
0      arancru@gmail.com                3
1    sofitor45@gmail.com                4
2  tamaiothais@gmail.com                2


In [5]:
# Dejar en el df df_horas solo los valores cuya action sea "peak"
df_horas = df_horas[df_horas['action'] == 'peak']

# a todos los valores sumarle 7 horas
df_horas['date'] = pd.to_datetime(df_horas['date'])
df_horas['date'] = df_horas['date'] + pd.Timedelta(hours=19)

In [6]:
# imprimir valores donde user_id es "tamaiothais@gmail.com"
print(df_horas[df_horas['user_id'] == 'tamaiothais@gmail.com'])

       app                             date                user_id action
780  swift 2024-11-01 22:01:47.294000+00:00  tamaiothais@gmail.com   peak
781  swift 2024-11-01 23:23:00.689000+00:00  tamaiothais@gmail.com   peak
782  swift 2024-11-01 00:00:00.688000+00:00  tamaiothais@gmail.com   peak
783  swift 2024-11-01 00:23:32.304000+00:00  tamaiothais@gmail.com   peak
784  swift 2024-11-05 14:23:50.452265+00:00  tamaiothais@gmail.com   peak
785  swift 2024-11-05 01:23:59.245000+00:00  tamaiothais@gmail.com   peak
786  swift 2024-11-05 02:24:11.545000+00:00  tamaiothais@gmail.com   peak
787  swift 2024-11-05 03:24:16.005000+00:00  tamaiothais@gmail.com   peak
788  swift 2024-11-05 04:24:19.329000+00:00  tamaiothais@gmail.com   peak
789  swift 2024-11-05 14:24:28.754993+00:00  tamaiothais@gmail.com   peak
790  swift 2024-11-05 07:24:33.910000+00:00  tamaiothais@gmail.com   peak
791  swift 2024-11-05 07:24:39.372000+00:00  tamaiothais@gmail.com   peak
792  swift 2024-11-05 08:24:52.237000+

In [7]:
# Dejar en la columna date solo la hora, minuto y segundo, recuerda que el formato actual de la fecha es datetime64[ns, UTC]
# Por ejemplo 2024-09-10 05:00:00.001000+00:00

df_horas['date'] = df_horas['date'].dt.hour

In [8]:
df_horas.head(10)

,app,date,user_id,action
759,flutter,11,juanjo.osorioc@gmail.com,peak
760,flutter,11,juanjo.osorioc@gmail.com,peak
761,flutter,15,juanjo.osorioc@gmail.com,peak
762,flutter,16,pauladaza600@gmail.com,peak
763,flutter,11,pauladaza600@gmail.com,peak
764,flutter,11,pauladaza600@gmail.com,peak
765,flutter,11,pauladaza600@gmail.com,peak
766,flutter,11,pauladaza600@gmail.com,peak
767,flutter,12,pauladaza600@gmail.com,peak
768,flutter,12,pauladaza600@gmail.com,peak


In [9]:
# Agrupar por hora y contar cuantas veces se repite
df_horas = df_horas.groupby('date').size().reset_index(name='count')
df_horas['date'] = df_horas['date'].apply(lambda x: f'{x}:00')

In [10]:
# Crear una columna que sea orden, donde el valor sea solo un numero
# 1:00 es 1, 2:00 es 2, etc
df_horas['orden'] = df_horas['date'].apply(lambda x: int(x.split(':')[0]))

In [11]:
df_horas.head(10)

,date,count,orden
0,0:00,3,0
1,1:00,1,1
2,2:00,1,2
3,3:00,1,3
4,4:00,1,4
5,7:00,2,7
6,8:00,1,8
7,11:00,8,11
8,12:00,4,12
9,13:00,4,13


In [12]:
# Cambiar los 0:00 por 24:00
df_horas['date'] = df_horas['date'].replace('0:00', '24:00')

# El 0 pasa a ser 24
df_horas['orden'] = df_horas['orden'].replace(0, 24)

### Type 2: How many days have passed since the user last contact a landlord?
### Type 3: How often has the user used the filter section in the past week?
### Type 3: What percentage of available properties are actively listed?
### Type 4: In which month of the year are the most offers published?
### Type *: Who are the top 3 most contacted landlords on the platform?

In [13]:
# Convertir la columna 'date' a datetime sin zona horaria
df['date'] = df['date'].dt.tz_localize(None)

In [14]:
# Extraer en un df solo las filas que tengan como valor de action 'contact'
contact_df = df[df['action'] == 'contact']

# Por cada user ID dejar solo el valor que tenga la fecha mas actual
contact_df = contact_df.sort_values('date', ascending=False).drop_duplicates('user_id')

# Calcular los dias que han pasado desde esa fecha hasta hoy
contact_df['days_since_contact'] = (pd.Timestamp.now() - contact_df['date']).dt.days

# Si el valor es -1 cambiarlo por 0
contact_df['days_since_contact'] = contact_df['days_since_contact'].apply(lambda x: 0 if x == -1 else x)

In [15]:
contact_df.head(10)

,app,date,user_id,action,days_since_contact
518,flutter,2024-11-04 18:07:25.618986,pauladaza600@gmail.com,contact,0
462,flutter,2024-11-03 16:21:19.799670,juanjo.osorioc@gmail.com,contact,0
702,swift,2024-11-01 08:36:24.518700,tamaiothais@gmail.com,contact,3
659,swift,2024-10-31 02:05:23.683058,s.torresr20@gmail.com,contact,4
448,swift,2024-10-30 15:09:29.120273,arancru@gmail.com,contact,4
661,swift,2024-09-02 17:53:20.456000,sofia.torres@plurall.com,contact,62


In [16]:
# Agregar la columna 'days_since_contact' al df original teniendo en cuenta el user_id
df = df.merge(contact_df[['user_id', 'days_since_contact']], on='user_id', how='left')

# Cualquier valor vacío en 'days_since_contact' cambiarlo por "NA"
df['days_since_contact'] = df['days_since_contact'].fillna('NA')

In [17]:
df.head(10)

,app,date,user_id,action,days_since_contact
0,swift,2024-09-10 05:00:00.001000,s.torresr20@gmail.com,contact,4.0
1,swift,2024-09-30 18:17:18.409000,arancru@gmail.com,filter,4.0
2,swift,2024-10-02 20:17:18.591994,arancru@gmail.com,filter,4.0
3,swift,2024-10-02 20:17:34.472924,arancru@gmail.com,filter,4.0
4,swift,2024-10-03 17:54:16.737926,arancru@gmail.com,filter,4.0
5,swift,2024-10-06 18:47:12.818138,arancru@gmail.com,filter,4.0
6,swift,2024-10-06 18:48:57.571789,arancru@gmail.com,filter,4.0
7,swift,2024-10-07 14:39:29.430546,arancru@gmail.com,filter,4.0
8,swift,2024-10-07 14:39:35.819237,arancru@gmail.com,filter,4.0
9,swift,2024-10-07 14:39:40.815232,arancru@gmail.com,filter,4.0


### Type 3: What percentage of property descriptions are left empty?
### Type 4: What are the top 5 most viewed properties?

In [18]:
# Acceder a la colección 'properties'
properties = db.collection('properties')

# Acceder a la colección 'offers'
offers = db.collection('offers')

# Obtener todos los documentos en la colección
docs_properties = properties.stream()
docs_offers = offers.stream()

In [19]:
# Crear una lista para almacenar los datos de cada propiedad
properties_data = []

# Recorrer cada documento en la colección 'properties'
for doc in docs_properties:
    # Extraer el contenido del documento como diccionario
    properties_dict = doc.to_dict()
    
    # Cada documento tiene un diccionario con múltiples propiedades, extraer cada propiedad
    for property_id, property_data in properties_dict.items():
        # Agregar el ID de la propiedad al diccionario de datos
        property_data['property_id'] = property_id
        # Añadir el diccionario de cada propiedad a la lista
        properties_data.append(property_data)

# Convertir la lista a un DataFrame de pandas
df_properties = pd.DataFrame(properties_data)

# Lo mismo para offers
offers_data = []

for doc in docs_offers:
    offers_dict = doc.to_dict()
    
    for offer_id, offer_data in offers_dict.items():
        offer_data['offer_id'] = offer_id
        offers_data.append(offer_data)

df_offers = pd.DataFrame(offers_data)

In [20]:
df_offers.head(10)

,price_per_month,initial_date,only_andes,final_date,views,num_baths,roommates,id_property,is_active,num_beds,user_id,num_rooms,type,offer_id
0,1000000.0,2024-12-04 17:55:00+00:00,False,2025-01-01 17:55:00+00:00,151,1,1,5,False,2,sofitor45@gmail.com,1,a_room,6
1,1800000.0,2024-09-04 17:36:00.998000+00:00,False,2025-06-01 17:36:00+00:00,93,1,0,4,True,1,sofitor45@gmail.com,1,a_room,5
2,3000000.0,2025-02-02 05:00:00.848000+00:00,False,2025-04-01 05:00:00.592000+00:00,247,5,2,2,True,2,sofitor45@gmail.com,2,a_room,2
3,1500000.0,2025-01-25 05:00:00.625000+00:00,True,2025-06-25 05:00:00.431000+00:00,101,1,0,1,True,4,arancru@gmail.com,4,entire_place,1
4,3172406.0,2024-11-01 06:57:53.627951+00:00,False,2024-11-08 06:57:53.627953+00:00,1,3,1,11,True,2,tamaiothais@gmail.com,2,a_room,12
5,2500000.0,2024-12-01 14:27:00+00:00,False,2025-01-31 14:27:00+00:00,65,1,0,3,True,1,sofitor45@gmail.com,1,entire_place,4
6,1000000.0,2024-11-01 06:57:16.764729+00:00,False,2024-11-08 06:57:16.764730+00:00,2,1,1,10,True,2,tamaiothais@gmail.com,2,a_room,11


In [21]:
df_properties.head(10)

,description,photos,complex_name,address,title,location,minutes_from_campus,property_id
0,,"[tamaiothais@gmail.com_10_1.jpg, tamaiothais@g...",Apartamento Cercano,Trv 517 #172-162,Apartamento Cercano,[],15,10
1,This spacious apartment in City U is shared wi...,"[apartment_image.jpg, cityu2.jpg]",City_U,"Ac. 19 #2a - 10, Bogotá",Apartment - T2 - 1102,<google.cloud.firestore_v1._helpers.GeoPoint o...,5,1
2,,"[sofitor45@gmail.com_5_1.jpg, sofitor45@gmail....",Bed in CentralX,Central X cra 3,Bed in CentralX,[],5,5
3,,"[livingx.jpg, living2.jpg]",Santa_fe,Calle 20 # 4 -34,House 5BR 1B,<google.cloud.firestore_v1._helpers.GeoPoint o...,10,2
4,I’m going on vacations so I want to sublet my ...,"[sofitor45@gmail.com_3_1.jpg, sofitor45@gmail....",Big apartment in city U,Cra 3 # 16a - 54,Big apartment in city U,[],5,3
5,I want to sublet a bedroom in a little apartme...,"[sofitor45@gmail.com_4_1.jpg, sofitor45@gmail....",A Bedroom in the spot,Cra 5 # 20 - 32,A Bedroom in the spot,[],7,4
6,Muy bonito y grande,"[tamaiothais@gmail.com_11_1.jpg, tamaiothais@g...",Apartamento estudiantil,Calle 1727 #1627-71,Apartamento estudiantil,[],23,11


In [22]:
# Rename en df_offers para que la columna 'id_property' sea 'property_id'
df_offers.rename(columns={'id_property': 'property_id'}, inplace = True)

# Asegúrate de que 'property_id' esté presente y sea consistente en ambos DataFrames
df_offers['property_id'] = df_offers['property_id'].astype(str)
df_properties['property_id'] = df_properties['property_id'].astype(str)

# Realizar el merge de df_offers con df_properties para agregar la descripción de la propiedad a la oferta
# 'property_id' será la columna de enlace
df_merged = df_offers.merge(df_properties[['property_id', 'description']], on='property_id', how='left')

# Teniendo en cuenta el user_id, agregar la columna 'title' del df_properties
df_merged = df_merged.merge(df_properties[['property_id', 'title']], on='property_id', how='left')

In [23]:
df_merged.head(10)

,price_per_month,initial_date,only_andes,final_date,views,num_baths,roommates,property_id,is_active,num_beds,user_id,num_rooms,type,offer_id,description,title
0,1000000.0,2024-12-04 17:55:00+00:00,False,2025-01-01 17:55:00+00:00,151,1,1,5,False,2,sofitor45@gmail.com,1,a_room,6,,Bed in CentralX
1,1800000.0,2024-09-04 17:36:00.998000+00:00,False,2025-06-01 17:36:00+00:00,93,1,0,4,True,1,sofitor45@gmail.com,1,a_room,5,I want to sublet a bedroom in a little apartme...,A Bedroom in the spot
2,3000000.0,2025-02-02 05:00:00.848000+00:00,False,2025-04-01 05:00:00.592000+00:00,247,5,2,2,True,2,sofitor45@gmail.com,2,a_room,2,,House 5BR 1B
3,1500000.0,2025-01-25 05:00:00.625000+00:00,True,2025-06-25 05:00:00.431000+00:00,101,1,0,1,True,4,arancru@gmail.com,4,entire_place,1,This spacious apartment in City U is shared wi...,Apartment - T2 - 1102
4,3172406.0,2024-11-01 06:57:53.627951+00:00,False,2024-11-08 06:57:53.627953+00:00,1,3,1,11,True,2,tamaiothais@gmail.com,2,a_room,12,Muy bonito y grande,Apartamento estudiantil
5,2500000.0,2024-12-01 14:27:00+00:00,False,2025-01-31 14:27:00+00:00,65,1,0,3,True,1,sofitor45@gmail.com,1,entire_place,4,I’m going on vacations so I want to sublet my ...,Big apartment in city U
6,1000000.0,2024-11-01 06:57:16.764729+00:00,False,2024-11-08 06:57:16.764730+00:00,2,1,1,10,True,2,tamaiothais@gmail.com,2,a_room,11,,Apartamento Cercano


In [24]:
# Solo necesito el user_id, la description, views, property_id y title
df_final = df_merged[['user_id', 'description', 'views', 'property_id', 'title', 'is_active']]

# Rename de user_id a landlord, description a descripcion, views a vistas, property_id a property
df_final.rename(columns={'user_id': 'landlord', 'description': 'descripcion', 'views': 'vistas', 'property_id': 'property'}, inplace = True)

/var/folders/57/132bv7p52t3c33gn6rjfp63h0000gn/T/ipykernel_55097/1169405012.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.rename(columns={'user_id': 'landlord', 'description': 'descripcion', 'views': 'vistas', 'property_id': 'property'}, inplace = True)


In [25]:
# Imprimir los valores unicos de "days_since_contact" en df

In [26]:
# Reemplazar las descripciones vacías con True y las llenas con False
df_final['descripcion'] = df_final['descripcion'].apply(lambda x: "Empty" if x == "" else "Complete")

# Cualquier valor que sea NA en la columna days_since_contact en df_final cambiarlo por 0
df['days_since_contact'] = df['days_since_contact'].apply(lambda x: 0 if x == 'NA' else x)

# En df_final cambiar los valores de la columna 'is_active' a 'Active' si es True y 'Inactive' si es False
df_final['is_active'] = df_final['is_active'].apply(lambda x: 'Active' if x == True else 'Inactive')

/var/folders/57/132bv7p52t3c33gn6rjfp63h0000gn/T/ipykernel_55097/3475180024.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['descripcion'] = df_final['descripcion'].apply(lambda x: "Empty" if x == "" else "Complete")
/var/folders/57/132bv7p52t3c33gn6rjfp63h0000gn/T/ipykernel_55097/3475180024.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['is_active'] = df_final['is_active'].apply(lambda x: 'Active' if x == True else 'Inactive')


In [27]:
# Guardar tambien como dos csv serparados por ,
df_final.to_csv('data_sources/ofertas.csv', index=False, sep=',')
df.to_csv('data_sources/user_actions.csv', index=False, sep=',')
df_horas.to_csv('data_sources/peak_hours.csv', index=False, sep=',')

In [28]:
# Definir el alcance de las credenciales de acceso a Google Sheets
scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

# Autorizar el cliente de gspread con las credenciales del archivo JSON
credentials = ServiceAccountCredentials.from_json_keyfile_name('credentials/client_secret.json', scope)
client = gspread.authorize(credentials)

# Abrir la hoja de cálculo de Google Sheets
spreadsheet = client.open('analyticsBQ')

In [29]:
# Crear una nueva hoja si no existe
def get_or_create_worksheet(spreadsheet, sheet_name, rows=100, cols=20):
    try:
        # Intentar obtener la hoja
        worksheet = spreadsheet.worksheet(sheet_name)
    except gspread.exceptions.WorksheetNotFound:
        # Crear la hoja si no existe
        worksheet = spreadsheet.add_worksheet(title=sheet_name, rows=rows, cols=cols)
    return worksheet

# Obtener o crear la hoja "User Actions"
worksheet_user_actions = get_or_create_worksheet(spreadsheet, "User Actions")

# Obtener o crear la hoja "Ofertas"
worksheet_ofertas = get_or_create_worksheet(spreadsheet, "Ofertas")

# Obtener o crear la hoja "Horas Pico"
worksheet_horas_pico = get_or_create_worksheet(spreadsheet, "Horas Pico")

In [30]:
# Leer y subir 'ofertas.csv' a la hoja "Ofertas"
df_ofertas = pd.read_csv('data_sources/ofertas.csv')
df_ofertas = df_ofertas.replace([float('inf'), float('-inf'), None], 0)  # Reemplazar valores infinitos con 0
df_ofertas = df_ofertas.fillna('')  # Reemplazar NaN con cadenas vacías
worksheet_ofertas.clear()  # Limpiar la hoja antes de subir los nuevos datos
worksheet_ofertas.update([df_ofertas.columns.values.tolist()] + df_ofertas.values.tolist())  # Subir el contenido

# Leer y subir 'user_actions.csv' a la hoja "User Actions"
df_user_actions = pd.read_csv('data_sources/user_actions.csv')
df_user_actions = df_user_actions.replace([float('inf'), float('-inf'), None], 0)  # Reemplazar valores infinitos con 0
df_user_actions = df_user_actions.fillna('')  # Reemplazar NaN con cadenas vacías
worksheet_user_actions.clear()  # Limpiar la hoja antes de subir los nuevos datos
worksheet_user_actions.update([df_user_actions.columns.values.tolist()] + df_user_actions.values.tolist())  # Subir el contenido

# Leer y subir 'peak_hours.csv' a la hoja "Horas Pico"
df_peak_hours = pd.read_csv('data_sources/peak_hours.csv')
df_peak_hours = df_peak_hours.replace([float('inf'), float('-inf'), None], 0)  # Reemplazar valores infinitos con 0
df_peak_hours = df_peak_hours.fillna('')  # Reemplazar NaN con cadenas vacías
worksheet_horas_pico.clear()  # Limpiar la hoja antes de subir los nuevos datos
worksheet_horas_pico.update([df_peak_hours.columns.values.tolist()] + df_peak_hours.values.tolist())  # Subir el contenido

print("Datos subidos exitosamente a las hojas 'User Actions', 'Ofertas' y 'Horas Pico' en Google Sheets.")

Datos subidos exitosamente a las hojas 'User Actions', 'Ofertas' y 'Horas Pico' en Google Sheets.
